This script calculates the distance of the safecast data points from every unique mext location

In [2]:
from math import cos, sqrt, sin, radians
import pandas as pd
import numpy as np
import pickle
import datetime
import dateutil.parser

mloc_hash = pickle.load(open('mloc_hash.pkl','rb')) #mext lat,long -> mloc_id
mloc_hash_rev = pickle.load(open('mloc_hash_rev.pkl','rb')) #mloc_id -> mext lat,long

mrad_hash = {} #mext lat,long in radians -> mloc_id
for key in mloc_hash:
    mrad_hash[tuple(map(radians,key))] = mloc_hash[key]
    
mext_rad = list(mrad_hash.keys())

def equirect_approx(lat1,lng1,lat2,lng2):
    """
    values must be in radians"""
    x = (lng2-lng1)*cos((lat1+lat2)/2.0)
    y = lat2-lat1
    return sqrt(x*x + y*y) * 6371.0 #earth radius in km

def haversine_distance(lat1,lng1,lat2,lng2):
    R = 6731.0 #radius of Earth in km
    dlon = lng2 - lng1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    c = 2 * np.arctan2(sqrt(a), sqrt(1-a)) 
    return 6731.0 * c

def dist_from_all_mext(latd,lngd):
    """
    inputs are in degrees"""
    
    lat,lng = map(radians, (latd,lngd))
#     return [round(equirect_approx(m[0],m[1],lat,lng),5) for m in mext_rad] #rounded to save disk space and time, equirect approximation
    return [round(haversine_distance(m[0],m[1],lat,lng),5) for m in mext_rad] #haversine formula
#     return [equirect_approx(m[0],m[1],lat,lng) for m in mext_rad] #unrounded

def min_distance(mext_distances):
    """
    returns the minimum distance from any point, lat of that point, long of that point
    """
    ind = np.argmin(mext_distances)
    mloc_id = mrad_hash[mext_rad[ind]]
#     return [mext_distances[ind],loc[0],loc[1]]
    loc = mloc_hash_rev[mloc_id]
    return [mext_distances[ind],loc[0],loc[1],mloc_id] #saves the mext location as a number corresponding to the index in mext_rad list object

mloc_times_hash = pickle.load(open('mloc_times_hash.pkl','rb')) #mext lat, long -> list of times

def closest_time(mloc_id,sftime):
    """
    returns the closest mext time for that mext location and the time difference from safecast time"""
    mloc = mloc_hash_rev[mloc_id]
    diff = []
    sftime = dateutil.parser.parse(sftime)
    for mtime in mloc_times_hash[mloc]:
        tdiff = sftime - mtime
        diff.append(abs(tdiff.total_seconds()))
#     print(diff)
#     print(np.argmin(diff),min(diff))
#     print(diff[890])
    close_time = mloc_times_hash[mloc][np.argmin(diff)]
    return [close_time,min(diff)]

In [73]:
#goes through the safecast data (time filtered) and calculates the closest point of all the mext points
#saves two separate files
#1. all the safecast data plus closest point information
#2. only closest point information

import timeit
import csv
from math import radians



def main():
    t = timeit.default_timer()
    with open('safecast2.csv','w') as wf:
        writer = csv.writer(wf)
        with open('safecast_data_datefiltered.csv','r') as sf:
            i=0
            #write header: distance from point at:, lat, long, mloc_id
            writer.writerow(next(sf).strip().split(',')+['distance','mlat','mlong','mloc_id','mtime','timediff','mdr','drdiff'])
            for line in sf:
                i+=1
                l=line.split(',')
                l[5] = l[5].strip()
    #             print(dist_from_all_mext(*map(float,(l[1:3])))[0])
                try:
    #                 writer.writerow(dist_from_all_mext(*map(float,(l[1:3])))) #only write distances
                    dist_lat_long = min_distance(dist_from_all_mext(*map(float,(l[1:3]))))
                    ctime= closest_time(dist_lat_long[-1],l[-1])
#                     mdr = mloctimes2dr[(dist_lat_long[1],dist_lat_long[2])][ctime[0]]
#                     drdiff = mdr - round(float(l[4]),5)
#                     writer.writerow(l+dist_lat_long+ctime+[mdr,drdiff])
                except ValueError:
                    raise
                    writer.writerow(['nan']*3)
                    dwriter.writerow(l+['nan']*3)
                break
                if i%100000==0:
#                 if i%1000==0:
                    print(i,timeit.default_timer()-t)
#                     break


In [5]:
#takes the safecast2.csv made above and adds on the mext doserate and doserate difference for the mext loc,time listed
#in each row of safecast2.csv

import timeit
import csv
import pickle
import dateutil.parser
mloctimes2dr = pickle.load(open('mloctimes2dr_hash.pkl','rb'))
t = timeit.default_timer()
with open('safecast3.csv','w') as wf:
    writer = csv.writer(wf)
    j=0
    with open('safecast2.csv','r') as rf:
        writer.writerow(next(rf).strip().split(',')+['mdr','drdiff'])
        for line in rf:
            l=line.split(',')
            l[-1] = l[-1].strip()
            j+=1
            mloc = (float(l[7]),float(l[8]))
            mtime = dateutil.parser.parse(l[10])
            mdr = mloctimes2dr[mloc][mtime]
            drdiff = mdr - float(l[4]) #'drdiff' column is mext doserate - safecast doserate
            writer.writerow(l+[mdr,round(drdiff,5)])
            if j%100000==0:
                print(j,timeit.default_timer()-t)
#                 break


100000 10.73643597587943
200000 21.4968832321465
300000 32.393808927387
400000 44.17645039781928
500000 55.42147144675255
600000 66.36579755321145
700000 77.37134133651853
800000 88.29467448219657
900000 99.24919581413269
1000000 110.24152001738548
1100000 121.70414825901389
1200000 133.03607867658138
1300000 145.10657830536366
1400000 156.76738172397017
1500000 167.5400036536157
1600000 179.1456928998232
1700000 190.79187834635377
1800000 201.35507857427
1900000 211.8077475465834
2000000 222.49953078851104
2100000 233.03681899607182


In [80]:
mloc

(35.706461, 139.697927)

In [74]:
main()

In [9]:
mtime - datetime.datetime(2011,11,2,4,0)

datetime.timedelta(-1, 82800)